# Error on Numerical Resolution of Systems of Linear Equation 

**TERMINOLOGY FOR THIS NOTEBOOK**
  + $A$ - Coefficient Matrix
  + $B$ - Constants Vector
  + $X$ - Solution Vector
  + $R$ - Residue Vector


## Fixed Significant Figures Arithmetic Operators

For this notebook, we will always want to perform the calculations on a fixed number of significant figures.

In a previous class, we discovered a Python library with a function that allowed us to get a value rounded to the given number of significant figures.
```
from sigfig import round
round(3.2467231, sigfigs = 3)
  >>> 3.25
round(3.2467231, sigfigs = 5)
  >>> 3.2467
```

To simplify our code, instead of calling multiple times ```round(value, sigfigs = number_significative_figures)```, we'll use a little simpler system.

We'll define functions ```add```, ```sub```, ```mul``` and ```div``` so that we can represent $ \frac{A \times B}{A + B} $ as
```
div(mul(A, B, sf=3), add(A, B, sf=3), sf=3)
```
instead of
```
round(round(round(A, sigfigs=3) * round(B, sigfigs=3), sigfgis=3) / round(round(A, sigfigs=3) + round(B, sigfigs=3), sigfgis=3), sigfigs=3)
```

In [2]:
from sigfig import round

SIGNIFICANT_FIGURES = 1

def add(x, y, sf=SIGNIFICANT_FIGURES):
    if sf == -1:
        return x+y
    x = round(x, sigfigs=sf)
    y = round(y, sigfigs=sf)
    return round(x+y, sigfigs=sf)

def sub(x, y, sf=SIGNIFICANT_FIGURES):
    if sf == -1:
        return x-y
    x = round(x, sigfigs=sf)
    y = round(y, sigfigs=sf)
    return round(x-y, sigfigs=sf)

def mul(x, y, sf=SIGNIFICANT_FIGURES):
    if sf == -1:
        return x*y
    x = round(x, sigfigs=sf)
    y = round(y, sigfigs=sf)
    return round(x*y, sigfigs=sf)

def div(x, y, sf=SIGNIFICANT_FIGURES):
    if sf == -1:
        return x/y
    x = round(x, sigfigs=sf)
    y = round(y, sigfigs=sf)
    return round(x/y, sigfigs=sf)

## Gaussian Elimination

We aim to perform Gaussian Elimination on the system of linear equations in order to obtain an upper traingular coefficient matrix.

In Gaussian Elimination, we want to find the system with an upper triangular coefficient matrix equivalent to the starting system.

To achieve that equivalent system, which will have an upper triangular coefficient matrix, we need to perform the Gaussian Elimination algorithm:

**_k-th step of the Gaussian Elimination algorithm_**

```
for i in [k+1, ..., n]
    if A[k][k] != 0
        m = A[i][k] / A[k][k]
        A[i] = A[i] - m * A[k]
        B[i] = B[i] - m * B[k]
```

Since ```A[i]``` represents the $i$-th row of $A$, then ```A[i] - m * A[k]``` is an element-wise difference.

Note that we only have to perform that difference on the elements that start on the same column as ```A[k][k]```.

Now, we only have to apply this algorithm to each element ```A[k][k]```.

### Implementation of the Gaussian Elimination algorithm

In [3]:
# functions for some prettier printing

def print_augmented_matrix(A, B):
    for i in range(len(A)):
        print(*A[i], "\t|\t", B[i])
    print()

def print_solution(X):
    for i in range(len(X)):
        print(f"x{i+1} = {X[i]}")

In [4]:
import numpy as np

# returns the upper triangular matrix resultant of the Gaussian Elimination method
def gaussian_elimination(A, B, sf):
    n = len(B)
    # pass the values to the correct number of significant figures, just in case
    for i in range(n):
        for j in range(n):
            A[i][j] = round(A[i][j], sigfigs=sf)
        B[i] = round(B[i], sigfigs=sf)
    # start the Gaussian Elimination algorithm
    for k in range(n):
        for i in range(k+1, n):
            m = div(A[i][k], A[k][k], sf)
            for j in range(k, n):
                A[i][j] = sub(A[i][j], mul(m, A[k][j], sf), sf)
            B[i] = sub(B[i], mul(m, B[k], sf), sf)

    return A, B

## Solving the system after Gaussian Elimination

Once we have the upper triangular coefficient matrix and the resulting constants vector, we can find the solution vector by solving for each variable with backwards stepping.

That is, since it is trivial to find the solution to the last variable, we can achieve previous solutions by using the already known ones.

The last variable is easy to calculate:
```
X[n] = B[n] / A[n][n]
```

The next ones we can do iteratively as:
```
for i in [n-1, ..., 1]
    X[i] = ( B[i] - sum(A[i][j] * X[j], for j in [i+1, ..., n]) ) / A[i][i] 
```

### Implementation of the algorithm to obtain the Solution Vector, $X$, given the results from the Gaussian Elimination algorithm

In [5]:
# solves the upper triangular matrix resultant of the Gaussian Elimination method
def solve_upper_triangular(A, B, sf):
    n = len(B)
    X = ["to be defined" for _ in range(n)]
    # pass the values to the correct number of significant figures, just in case
    for i in range(n):
        for j in range(n):
            A[i][j] = round(A[i][j], sigfigs=sf)
        B[i] = round(B[i], sigfigs=sf)
    # backword stepping to calculate the variables
    for i in reversed(range(n)):
        # get the last variable
        if i == n-1:
            X[i] = div(B[i], A[i][i], sf)
            continue
        # get the remaining variables
        X[i] = B[i]
        for j in range(i+1, n):
            X[i] = sub(X[i], mul(A[i][j], X[j], sf), sf)
        X[i] = div(X[i], A[i][i], sf)

    return X

### Testing with theoretical class example

In [6]:
A = [
    [-1.41, 2, 0],
    [1, -1.41, 1],
    [0, 2, -1.41]
]

B = [1, 1, 1]

a1, b1 = gaussian_elimination(A, B, 3)
print_augmented_matrix(a1, b1)
X = solve_upper_triangular(a1, b1, 3)
print_solution(X)


-1.41 2 0 	|	 1
0.0 0.01 1.0 	|	 1.71
0.0 0.0 -201.0 	|	 -341.0

x1 = 0.709
x2 = 1.0
x3 = 1.7


d:\anaconda\lib\site-packages\sigfig\sigfig.py:586: UserWarning: warning: 3 significant figures requested from number with only 1 significant figures
  warn("warning: %d significant figures requested from number with only %d significant figures" % (given['sigfigs'], len(num.map)))
d:\anaconda\lib\site-packages\sigfig\sigfig.py:586: UserWarning: warning: 3 significant figures requested from number with only 2 significant figures
  warn("warning: %d significant figures requested from number with only %d significant figures" % (given['sigfigs'], len(num.map)))


## Propagation of the Error

After calculating the Solution Vector, $X$, we want to check how much the error propagfrom the limited significant figures system.

Let the Residue Vector be $R = AX-B$.

The squared value of the magnitude of the Residue Vector, $|R|^2$, gives a good measure of the amount of error propagated into the solution.

In [7]:
def get_residue_squared_magnitude(A, B, X, sf):
    # pass the values to the correct number of significant figures, just in case
    n = len(A)
    for i in range(n):
        for j in range(n):
            A[i][j] = round(A[i][j], sigfigs=sf)
        B[i] = round(B[i], sigfigs=sf)
        X[i] = round(X[i], sigfigs=sf)
    
    R = [0 for _ in range(len(X))]
    for r in range(len(R)):
        for j in range(len(A)):
            R[r] = add(R[r], mul(A[r][j], X[j], sf), sf)
        R[r] = sub(R[r], B[r], sf)

    error_prop = 0
    for r in R:
        error_prop = add(error_prop, mul(r, r, sf), sf)
    return error_prop

A = [
    [-1.41, 2, 0],
    [1, -1.41, 1],
    [0, 2, -1.41]
]

B = [1, 1, 1]

print(get_residue_squared_magnitude(A, B, X, 3))

1.96


## Solving Systems of Linear Equations with Partial Pivoting

The error propagates due to the choice of the pivot on the k-th steps of the Gaussian Elimination algorithm.

In Gaussian Elimination with **Partial Pivoting**, on the k-th step, we search for the highest value in the column $k$ and rows $k$ to $n$.

Let's say that value is in the $r$-th row. Then, we swap row $r$ with row $k$, and proceed with the remaining execution of the algorithm step.

### Implementation of the Gaussian Elimination algorith with Partial Pivoting

In [8]:
# returns the upper triangular matrix resultant of the Gaussian Elimination method
def gaussian_elimination_partial_pivoting(A, B, sf):
    n = len(B)
    # pass the values to the correct number of significant figures, just in case
    for i in range(n):
        for j in range(n):
            A[i][j] = round(A[i][j], sigfigs=sf)
        B[i] = round(B[i], sigfigs=sf)
    # start the Gaussian Elimination algorithm
    for k in range(n):
        # choose pivot
        r = k
        for i in range(k, n):
            if A[i][k] > A[r][k]:
                r = i
        A[r], A[k] = A[k], A[r]
        B[r], B[k] = B[k], B[r]
        # continue the algorithm
        for i in range(k+1, n):
            m = div(A[i][k], A[k][k], sf)
            for j in range(k, n):
                A[i][j] = sub(A[i][j], mul(m, A[k][j], sf), sf)
            B[i] = sub(B[i], mul(m, B[k], sf), sf)

    return A, B

A = [
    [-1.41, 2, 0],
    [1, -1.41, 1],
    [0, 2, -1.41]
]

B = [1, 1, 1]

a1, b1 = gaussian_elimination_partial_pivoting(A, B, 3)
X = solve_upper_triangular(a1, b1, 3)
print_solution(X)
print(f"Squared magnitude of the Residue Vector = {get_residue_squared_magnitude(A, B, X, 3)}")

x1 = 1.7
x2 = 1.7
x3 = 1.7
Squared magnitude of the Residue Vector = 0.0


Notebook criado por António Cardoso em 2023 para a Unidade Curricular "Métodos Numéricos (M2039)" na Faculdade de Ciências da Universidade do Porto.